In [1]:
import os
import json
import pandas as pd

In [2]:
fold = r"../data"

data = []

for file in os.listdir(fold):
    if not file.startswith("新一代信息技术产业集群"):
        continue
    file_name = os.path.join(fold, file)
    data.append(pd.read_excel(file_name))

df = pd.concat(data)

In [3]:
df.shape

(24451, 38)

In [4]:
df.to_csv("../data/2.5w_新一代信息技术企业.csv", index=False)

In [5]:
df.columns

Index(['标记', '企业名称', '所属区县', '登记状态', '法定代表人', '注册资本', '成立日期', '统一社会信用代码',
       '注册地址', '注册地址邮编', '有效手机号', '更多电话', '邮箱', '所属省份', '所属城市', '纳税人识别号',
       '注册号', '组织机构代码', '参保人数', '参保人数所属年报', '企业(机构)类型', '企业规模', '营业期限',
       '国标行业门类', '国标行业大类', '国标行业中类', '国标行业小类', '曾用名', '英文名', '官网网址', '通信地址',
       '通信地址邮编', '企业简介', '经营范围', '企查查行业门类', '企查查行业大类', '企查查行业中类', '企查查行业小类'],
      dtype='object')

In [6]:
def load_json(file):
    with open(file, "r", encoding="utf-8") as f:
        return json.load(f)

In [7]:
baidu_map_json_data = load_json("../API_results/2w_企业名称.json")

In [8]:
def get_lng_lat(address):
    res = baidu_map_json_data.get(address, {})
    if not res:
        return pd.NA, pd.NA
    obj = res["result"]["location"]
    return obj["lng"], obj["lat"]

In [9]:
get_lng_lat("合肥高尔生命健康科学研究院有限公司")

(117.08092673854266, 31.818082240214483)

In [10]:
baidu_map_json_data

{'合肥高尔生命健康科学研究院有限公司': {'status': 0,
  'result': {'location': {'lng': 117.08092673854266,
    'lat': 31.818082240214483},
   'precise': 1,
   'confidence': 70,
   'comprehension': 100,
   'level': '教育'},
  'idx': 5279,
  'address_input': '合肥高尔生命健康科学研究院有限公司'},
 '上海聚货通电子商务有限公司': {'status': 0,
  'result': {'location': {'lng': 106.0926076824792, 'lat': 29.863231290719845},
   'precise': 0,
   'confidence': 50,
   'comprehension': 0,
   'level': 'NoClass'},
  'idx': 13781,
  'address_input': '上海聚货通电子商务有限公司'},
 '惠州市方能精密电子有限公司': {'status': 0,
  'result': {'location': {'lng': 114.1317573056666, 'lat': 23.159208703625605},
   'precise': 0,
   'confidence': 50,
   'comprehension': 100,
   'level': 'NoClass'},
  'idx': 22283,
  'address_input': '惠州市方能精密电子有限公司'},
 '四川昆仑联合科技有限公司': {'status': 0,
  'result': {'location': {'lng': 104.25174129438595,
    'lat': 30.576759941354617},
   'precise': 0,
   'confidence': 50,
   'comprehension': 0,
   'level': 'NoClass'},
  'idx': 5280,
  'address_input': '四川昆

## 添加经纬度

In [11]:
def add_lng_lat(row):
    company_name = row["企业名称"]
    lng, lat = get_lng_lat(company_name)
    return pd.Series({"经度": lng, "纬度": lat})

In [12]:
df.apply(add_lng_lat, axis=1).shape

(24451, 2)

In [13]:
df.shape

(24451, 38)

In [14]:
df[["经度", "纬度"]] = df.apply(add_lng_lat, axis=1)

In [15]:
df.shape

(24451, 40)

In [16]:
df.to_csv("../output/2.5w_新一代信息技术企业经纬度_test.csv", index=False)

In [ ]:
df.to_excel("../output/2.5w_新一代信息技术企业经纬度.xlsx", index=False)